<a href="https://colab.research.google.com/github/Praks21/PestiCide_Residue_Analysis/blob/main/Pesticide_Residue_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the local paths for train and test datasets
train_dataset_path = r'C:\Users\prakh\Downloads\Vegetable dataset\train'
test_dataset_path = r'C:\Users\prakh\Downloads\Vegetable dataset\test'

In [ ]:
# Check if the paths exist
print("Train dataset path exists:", os.path.exists(train_dataset_path))
print("Test dataset path exists:", os.path.exists(test_dataset_path))


In [ ]:
# Initialize ImageDataGenerator for train and test data
datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Load the images from the train directory
train_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),  # Resize images to (224, 224) for the model
    batch_size=32,
    class_mode='categorical', # For multi-class classification
    shuffle=True  # Shuffle the dataset for training
)

In [ ]:
# Load the images from the test directory
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),  # Resize images to (224, 224) for the model
    batch_size=32,
    class_mode='categorical', # For multi-class classification
    shuffle=False  # Don't shuffle the test data
)

In [ ]:
# Example: Print out class labels
print("Classes found:", train_generator.class_indices)

In [ ]:
# Define the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the layers of ResNet50
for layer in base_model.layers:
    layer.trainable = False

# Add a custom top layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(3, activation='softmax')(x)  # 3 classes (change if necessary)

In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=2,  # Reduce epochs for testing, adjust as needed
    validation_data=test_generator
)

In [ ]:
# Save the trained model
model.save('vegetable_classification_model.h5')

In [ ]:
# Print final accuracy
print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")

In [ ]:
# Plotting the training and validation accuracy
plt.figure(figsize=(12, 6))

In [ ]:
# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.show()